In [2]:
from sympy import *

# Vertex transform maths and Computer Vision
OpenGL and similar graphics libraries don't work with projection matrices that
directly map vertices to pixels, but go through a few different transforms to
aide processing. In the OpenGL specification, the coordinate systems are
typically named World, Eye, Clip, NDC (normalized device coordinates) and
Window.

As computer vision practitioners, World is familiar, Eye is what we would often
call Camera, and Window is more or less image space pixels (with some caveats).
The purpose of these coordinate systems is to aide in geometry culling and
remapping depth values for the purposes of getting more our of finite computer
numerical precision when testing which geometry should be shown in front of each
other.

We'll introduce some new frames of reference which will allow us to interop with
computer vision notation and also provide us with a uniform treatment of
considering sub-views of images rendered with a combination of image space and
scene space primitives

### World coordinates:
$P_w = (x,y,z,1)^T \in R^4$ relative to world frame, homogeneous

### Camera coordinates:
$P_c = (x,y,z,1)^T \in R^4$ relative to camera frame, homogeneous
$$P_c = T_{cw} * P_w$$ 

### Calibrated coordinates:
$P_cal = (z*u, z*v, z, 1)^T$ depth-scaled pixel units
$$P_{cal} = \text{makeK4}(K) * P_c$$

### Pixel coordinates:
$P_pix = (z*u, z*v, z*Z_ndc, z)$ pixel-unit homogeneous
$$P_pix = \text{makePerspectiveNearFar}(near,far) * P_cal$$

### Clip coordinates:
$P_clip = (z*X_ndc, z*Y_ndc, z*Z_ndc, z)$, X,Y scaled to [-1,1] interval
$$P_clip = \text{makeClip}(w,h) * P_pix$$

Optionally, insert makeClipView(zoom,offset) for viewing transform in clip space
P_clip = makeClipView(zoom,offset) * makeClip(w,h) *
              makePerspectiveNearFar(near,far) * makeK4(K) * T_cw * P_w

In [31]:
W,H,N,F,fu,fv,u0,v0,x = symbols("W,H,N,F,fu,fv,u0,v0,x")
T_ba = MatrixSymbol('Tba', 4, 4)
P_a = MatrixSymbol('Pa', 4, 1)
P_b = T_ba * P_a

# Homogeneous Vision-style projection matrix
def makeK4(K):
    K4 = eye(4)
    K4[0:3,0:3] = K
    return K4

# Perspective matrix which copies .z into .w for later homogenous division, and
# scales .z into ndc depth range [-1, 1]. This is not the GL Projection matrix,
# which also would include mapping .x and .y into ndc range.
#     * +z is forward
#     * z_ndc=-1 maps to near plane
#     * z_ndc=+1 maps to far plane
#     * near and far should be same sign (corresponding to positive or negative z-axis as forward)
#     * far can be +/- oo (in which case limit is taken as far approaches oo)

#     * NOTE: Because of the non-linearity of floating point, near,far precision
#             isn't as trivial as you might think. Using an infinite far plane
#             is actually better almost throughout the entire range, beside a
#             tiny region on the near plane (nano-meters if the near plane is at
#             a meter, according to paper cited below), because the densely
#             represented region around (floating point) 0 is brought out into a
#             more usable range.

#   transforms (x,y, z,1) point in camera coordinates into 
#              (x,y,zn,z), where zn is ndc depth
def makePerspectiveNearFar(near, far):
    # https://terathon.com/gdc07_lengyel.pdf
    # https://developer.nvidia.com/content/depth-precision-visualized
    # http://www.geometry.caltech.edu/pubs/UD12.pdf
    # "Tightening the Precision of Perspective Rendering" Upchurch and Desbrun
    eps = 0
    nofar = abs(far) is oo
    P4 = eye(4)
    P4[2,2] = sign(far) if nofar else (far+near) / (far - near)
    P4[2,3] = -sign(far)*2*near if nofar else 2*far*near / (near - far)
    P4[3,3] = 0
    P4[3,2] = 1
    return P4

def makeOrthoNearFar(near,far):
    # limits don't work to eliminate far for ortho matrix :(
    P4 = eye(4)
    P4[2,2] = 2/(far-near)
    P4[2,3] = -(far+near) / (far-near)
    return P4

def makeZoomPan(scale, offset):
    ZP4 = eye(4)
    ZP4[0,0] = scale
    ZP4[1,1] = scale
    ZP4[0,3] = offset/scale
    ZP4[1,3] = offset/scale
    return ZP4

# Maps .x and .y in pixel units (either discrete or continous convention) to
# NDC [-1,1] interval.
# For cont=false, assumes pixel-centered input, where (-0.5,-0.5) is top-left of
# top-left pixel. For cont=true, (0,0) is top-left of top-left pixel
def makeClipFromPixel(W, H, cont = false):
    C4 = eye(4)
    C4[0,0] = 2/W
    C4[1,1] = 2/H
    C4[0,3] = -1 if cont else -(1-1/W)
    C4[1,3] = -1 if cont else -(1-1/H)
    return C4

# pre-multiply clip matrix to flip y top/bottom
def makeClipFlip():
    return diag(1,-1,1,1)

# Performs homogeneous division by .w for final NDC coordinates in
# [-1,-1,-1] to [+1,+1,+1] interval
# Note - NDC is further transformed to window coordinates
def toNdc(P_clip):
    return P_clip[0:3] / P_clip[3]

# Matrix maps NDC coordinates to Window coordinates given viewport
def toWindowCoordinates(x0, y0, w, h):
    pass

## Demonstration of typical near=>far Projection and near=>oo projection

In [14]:
K = Matrix([[fu,0,u0],[0,fv,v0],[0,0,1]])
K4 = makeK4(K)
P4 = makePerspectiveNearFar(N,F)
P4_inf = makePerspectiveNearFar(N,oo)
C4 = makeClipFromPixel(W,H)

display("K", K4)
display("Projection", P4, C4*P4*K4)
display("Projection infinite far", P4_inf, C4*P4_inf*K4)


'K'

Matrix([
[fu,  0, u0, 0],
[ 0, fv, v0, 0],
[ 0,  0,  1, 0],
[ 0,  0,  0, 1]])

'Projection'

Matrix([
[1, 0,               0,              0],
[0, 1,               0,              0],
[0, 0, (F + N)/(F - N), 2*F*N/(-F + N)],
[0, 0,               1,              0]])

Matrix([
[2*fu/W,      0, -1 + 2*u0/W + 1/W,              0],
[     0, 2*fv/H, -1 + 2*v0/H + 1/H,              0],
[     0,      0,   (F + N)/(F - N), 2*F*N/(-F + N)],
[     0,      0,                 1,              0]])

'Projection infinite far'

Matrix([
[1, 0, 0,    0],
[0, 1, 0,    0],
[0, 0, 1, -2*N],
[0, 0, 1,    0]])

Matrix([
[2*fu/W,      0, -1 + 2*u0/W + 1/W,    0],
[     0, 2*fv/H, -1 + 2*v0/H + 1/H,    0],
[     0,      0,                 1, -2*N],
[     0,      0,                 1,    0]])

## Verify how z value is mapped

In [6]:
def map_near_far(z_in, N, F):
    P4 = makePerspectiveNearFar(N,F)
    P_clip = P4 * Matrix([0,0,z_in,1])
    return simplify(P_clip[2] / P_clip[3])

def map_near_far_inf(z_in, N):
    P4 = makePerspectiveNearFar(N,oo)
    P_clip = P4 * Matrix([0,0,z_in,1])
    return simplify(P_clip[2] / P_clip[3])

print("For N=>F projection, Z=N maps to Z_ndc=", map_near_far(N, N, F))
print("For N=>F projection, Z=F maps to Z_ndc=", map_near_far(F, N, F))
print("For N=>F projection, Z=2*N maps to Z_ndc=", map_near_far(10*N, N, F))

print("For N=>oo projection, Z=N maps to Z_ndc=", map_near_far_inf(N, N))
print("For N=>oo projection, Z=oo maps to Z_ndc=", map_near_far_inf(oo, N))
print("For N=>F projection, Z=2*N maps to Z_ndc=", map_near_far_inf(10*N, N))


For N=>F projection, Z=N maps to Z_ndc= -1
For N=>F projection, Z=F maps to Z_ndc= 1
For N=>F projection, Z=2*N maps to Z_ndc= (4*F/5 + N)/(F - N)
For N=>oo projection, Z=N maps to Z_ndc= -1
For N=>oo projection, Z=oo maps to Z_ndc= 0
For N=>F projection, Z=2*N maps to Z_ndc= 4/5


## Check Perspective

In [7]:
# discrete pixel convention
K = Matrix([[fu,0,u0],[0,fv,v0],[0,0,1]])
P = makeClipFlip() * makeClipFromPixel(W,H,false) * makePerspectiveNearFar(N,oo)
tl = Matrix([-0.5*N,-0.5*N,N,1])
br = Matrix([(W-0.5)*N,(H-0.5)*N,N,1])
brz = Matrix([3*(W-0.5)*N,3*(H-0.5)*N,3*N,1])
tl_ndc = P*tl
br_ndc = P*br
brz_ndc = P*brz

# Make sure we can hit corners of NDC
display(simplify(tl_ndc/tl_ndc[3]))
display(simplify(br_ndc/br_ndc[3]))
# Make sure depth doesn't take us of NDC edge in x and y
display(simplify(brz_ndc/brz_ndc[3]))

Matrix([
[-1],
[ 1],
[-1],
[ 1]])

Matrix([
[ 1],
[-1],
[-1],
[ 1]])

Matrix([
[  1],
[ -1],
[1/3],
[  1]])

In [8]:
# continuous pixel convention
K = Matrix([[fu,0,u0],[0,fv,v0],[0,0,1]])
P = makeClipFromPixel(W,H,true) * makePerspectiveNearFar(N,oo)
tl = Matrix([0,0,N,1])
br = Matrix([W*N,H*N,N,1])
tl_ndc = P*tl
br_ndc = P*br
# Make sure we can hit corners of NDC
display(simplify(tl_ndc/tl_ndc[3]))
display(simplify(br_ndc/br_ndc[3]))

Matrix([
[-1],
[-1],
[-1],
[ 1]])

Matrix([
[ 1],
[ 1],
[-1],
[ 1]])

## Check Ortho

In [34]:
P = makeOrthoNearFar(N,F)

a = Matrix([0,0,N,1])
b = Matrix([5,5,N,1])
c = Matrix([5,5,2*N,1])

a_ndc = P*a
b_ndc = P*b
c_ndc = P*c

display(simplify(a_ndc/a_ndc[3]))
display(simplify(b_ndc/b_ndc[3]))
display(simplify(c_ndc/c_ndc[3]))

Matrix([
[ 0],
[ 0],
[-1],
[ 1]])

Matrix([
[ 5],
[ 5],
[-1],
[ 1]])

Matrix([
[ 5],
[ 5],
[-1],
[ 1]])

In [38]:
makeClipFromPixel(W,H,true) * makePerspectiveNearFar(N,oo) * makeK4(K)

Matrix([
[2*fu/W,      0, -1 + 2*u0/W,    0],
[     0, 2*fv/H, -1 + 2*v0/H,    0],
[     0,      0,           1, -2*N],
[     0,      0,           1,    0]])

In [39]:
makeClipFromPixel(W,H,true) * makeK4(K) * makePerspectiveNearFar(N,oo)

Matrix([
[2*fu/W,      0, -1 + 2*u0/W, -4*N*u0/W],
[     0, 2*fv/H, -1 + 2*v0/H, -4*N*v0/H],
[     0,      0,           1,      -2*N],
[     0,      0,           1,         0]])